1. Загрузить файл длиной не менее 2000 символов.

In [ ]:
# open file, read file, get length, print length, return file content
def load_file(filepath: str):
    with open(filepath, 'r') as f:
        file_content = f.read()
        symbols_count = len(file_content)
        print(f'File contains {symbols_count} symbols')
    
    return file_content

In [ ]:
filepath = 'text-file.txt'
file_text = load_file(filepath)


2. Составить программу, которая считает число уникальных слов в тексте (без критерия
схожести)

In [ ]:
# function to exclude non-akphabetic and non-spaces symbols 
from string import punctuation

# check that symbol is not punctuation sign
def is_target_symbol(symbol: str):
    return symbol not in punctuation

# exclude unnecessary symbols from text, split to words, make set of words, get set length
def calc_unique_words(text: str):
    words = ''.join(filter(is_target_symbol, text.lower())).split()
    uniques_count = len(set(words))
    print(f'There are {uniques_count} unique words in text')
    
    return uniques_count


In [ ]:
uniques_count = calc_unique_words(file_text)


3. Составить программу, которая считает число гласных и согласных букв.

In [ ]:
# iterate through string and check if symbol in vowels string
# if symbol is non-vowel but alphabetic - it's consonant
def calculate_letters(string: str):
    vowels = 'aeiou'
    vowels_count, consonants_count = 0, 0
    
    for symbol in string:
        if symbol in vowels or symbol.lower() in vowels:
            vowels_count += 1
        elif symbol.isalpha():
            consonants_count += 1
    
    return vowels_count, consonants_count


In [ ]:
vowels, consonants = calculate_letters(file_text)
print(f'Vowels count - {vowels}, consonats count - {consonants}')


4. Составить программу, которая считает число предложений, их длину и число (количество)
раз использования каждого слова в тексте (с критерием схожести, критерий схожести слов
выбрать самостоятельно, например, spacy (en_core_web_sm) или расстояние
Левенштейна).

Для подсчета предложений в тексте убираются точки из распространенных сокращений "Mrs.", "Mr." Заменяются другие знаки окончания предложения ("!", "?") на точку для упрощения разделения текста на предложения. Текст режется на предложения по разделителю ".", предложения делятся на слова. Для предложений, содержащих слова, выводится кол-во слов.

In [ ]:
# calculate count of sentences and words in each one
def proc_sentences(string: str):
    # replace reduction to exclide point
    for reduction in ['Mrs.', 'Mr.']:
        proc_str = string.replace(reduction, reduction[:-1])
    
    # replace other sentences separators to simplify spliting by point
    for separator in ['!', '?']:
        proc_str = proc_str.replace(separator, '.')
        
    sentences_list = proc_str.split('.')
    sentences_count = 0
    
    for item in sentences_list:
        item = item.strip()
        
        if item:        
            sentences_count += 1
            sentence_length = len(item.split())
            print('Sentence:\n', item, '\nWords count -', sentence_length, '\n\n')
    
    print('Sentences count -', sentences_count)
    

Для подсчета уникальных слов в тексте с критерием схожести сначала убираются знаки пунктуации. Текст (в нижнем регистре) разделяется на токены с помощью библиотеки spacy (легкая моделька en_core_web_sm). Составляется словарь: в качестве ключей - слова в первоначальной форме, значений - кол-ва вхождений слов-ключей в текст. Проверяется выделение символов '\n', '\t', '\s' в отдельный токен с последующем его игнорированием. 


In [ ]:
from string import punctuation, whitespace
import time
import spacy

# find unique words count with spacy
def spacy_algorithm(text: str):
    time1 = time.time()

    translator = str.maketrans('', '', punctuation)
    npl = spacy.load('en_core_web_sm')
    # replace punctuation symbols in lowercase text, create tokens from text
    tokens = npl(text.lower().translate(translator))

    unique_text_words = dict()
    # fill dictionary with unique words, if word is't in word separators
    for token in tokens:
        if token.text not in whitespace:
            unique_text_words[token.lemma_] = unique_text_words[token.lemma_] + 1 if token.lemma_ in unique_text_words.keys() else 1

    unique_text_words = dict(sorted(unique_text_words.items(), key=lambda item: item[1], reverse=True))
    print(f'Unique words count is {len(unique_text_words)}')

    print('Words used in text:')
    for item in unique_text_words.items():
        print(item)
    
    # get method time complexity in ms
    print('Time complexity', (time.time() - time1) * 1000, 'ms')
    return unique_text_words


In [ ]:
# file_text = load_file(filepath)
# proc_sentences(file_text)


In [ ]:
file_text = load_file(filepath)
spacy_unique_words = spacy_algorithm(file_text)


Как видно, среди часто встречаемых в тексте много слоВ, не несущих смысловой нагрузки, удаялем их из первой десятки часто встречаемых в тексте

In [ ]:
TOP_COUNT = 10

top_counter = 0
keys_to_delete = list()

print(TOP_COUNT, 'words frequently used in text:')
# check word key in dictionary is in stop words, append stop words to list, than remove from dictionary
for word_key in spacy_unique_words.keys():
    if word_key in spacy.lang.en.stop_words.STOP_WORDS:
        keys_to_delete.append(word_key)
        continue
    
    top_counter += 1
    print(word_key, spacy_unique_words[word_key])
    if top_counter == TOP_COUNT:
        break

for del_key in keys_to_delete:
    spacy_unique_words.pop(del_key)

print(len(keys_to_delete), 'stop words were excluded')
